In [52]:
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
import seaborn as sns
import sys
import sys
import re
import os.path

## data import

In [53]:
file_name =  "clean_data_2022-02-24.csv"
folder = "../clean_equity_data/"
data_tmp = pd.read_csv(folder + file_name)

In [54]:
# FUNCTIONS FOR VIEWING SINGLE COMPANY RANKINGS 

def company(company, data):
    for i in data['Company']:
        if company in i or company in i.lower():
            return data.loc[data['Company'] == i]
    print('No company found')
    
def Company(company, data):
    for i in data['Company']:
        if company in i or company in i.lower():
            stock = data.loc[data['Company'] == i]
            stock = stock[compact]
            return stock
    print('No company found')

In [55]:
### MOMENTUM
data_mom = data_tmp.copy()
#data_mom = data_mom.loc[(data_mom['Industry'] != 'Holding Companies')]


### GESTALT
data_gestalt = data_tmp.copy()
data_gestalt = data_gestalt.loc[(data_tmp['Sector'] != 'Financials')]

### Momentum Rank

In [56]:
# RANK ON DIFFERENT METRICS 

data_mom['3m Rank'] = data_mom['Return 3m'].rank(ascending = False)
data_mom['6m Rank'] = data_mom['Return 6m'].rank(ascending = False)
data_mom['1y Rank'] = data_mom['Return 1y'].rank(ascending = False)
data_mom['EA Rank'] = data_mom['EA ret'].rank(ascending = False)
data_mom['EA Std Rank'] = data_mom['EA ret std'].rank(ascending = False)
data_mom['Volatility Rank'] = data_mom['1 Year Volatility'].rank(ascending = True)
data_mom['MAD Rank'] = data_mom['1 Year Volatility'].rank(ascending = True)

# MOMENTUM WITH WOLATILITY AND EA
data_mom['Momentum Rank'] = (data_mom['3m Rank'] + data_mom['6m Rank'] +  data_mom['1y Rank'] + data_mom['Volatility Rank'] + data_mom['EA Std Rank']).rank(ascending = True)



### Gestalt Imputation and Rank

In [57]:
# MANAGE NaNs and NEGATIVE EARNINGS  

# set nan to median, only fundamental data
columns = ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B', 'ROC', 'ROE',
            'GPA', 'Assets Turn', 'FCFROE', 'Asset Growth']
for i in columns: 
    data_gestalt.loc[data_gestalt[i].isna() ,i] = data_gestalt[i].median()


    
# set negative values to max
for i in ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B']:
    data_gestalt.loc[data_gestalt[i] < 0 ,i] = data_gestalt[i].max()
    
# set nan yield & vol to 0
data_gestalt.loc[data_gestalt['Yield'].isna(),'Yield'] = 0

In [75]:
RANK_WEIGHTS = {"Momentum": 1, "Value": 1, "Quality": 1}


factor=1.0/sum(RANK_WEIGHTS.values())
for k in RANK_WEIGHTS:
    RANK_WEIGHTS[k] = RANK_WEIGHTS[k]*factor
    
    
print(sum(RANK_WEIGHTS.values()))



In [77]:
####### CALCULATE RANKS ###########
for i in ['Return 3m', 'Return 6m', 'Return 1y', 'EA ret std']:
    data_gestalt[i +' Rank'] = data_gestalt[i].rank(ascending = False)

# Ranking where lower value is better
for i in ['P/E', 'P/B', 'P/S', 'P/FCF', 'EV/EBIT', 'Asset Growth']:
    data_gestalt[i +' Rank'] = data_gestalt[i].rank()
    

for i in ['Yield', 'ROE', 'ROC', 'FCFROE', 'GPA', 'Assets Turn']:
    data_gestalt[i + ' Rank'] = (data_gestalt[i]).rank(ascending = False)
    
##################### Composite ranks #######################

data_gestalt['Momentum Rank'] = (data_gestalt['Return 3m Rank'] + data_gestalt['Return 6m Rank'] + 
                                 data_gestalt['Return 1y Rank'] + data_gestalt['EA ret std Rank']).rank(ascending = True)


data_gestalt['Quality Rank'] = (data_gestalt['ROE Rank'] + data_gestalt['ROC Rank'] + data_gestalt['FCFROE Rank'] + 
                        data_gestalt['GPA Rank'] + data_gestalt['Assets Turn Rank']).rank()

data_gestalt['Value Rank'] = (data_gestalt['P/E Rank'] + data_gestalt['P/B Rank'] + data_gestalt['P/S Rank'] + data_gestalt['P/FCF Rank'] + 
                      data_gestalt['EV/EBIT Rank'] + data_gestalt['Asset Growth Rank']).rank()
 


    
data_gestalt['Gestalt Rank'] = ((RANK_WEIGHTS['Value']*data_gestalt['Value Rank'] 
                                 + RANK_WEIGHTS['Momentum']*data_gestalt['Momentum Rank'] +
                                RANK_WEIGHTS['Quality']* data_gestalt['Quality Rank'])).rank()



### Data Error Check

In [65]:
#data_mom.isna().sum()
data_gestalt.isna().sum()

Unnamed: 0            0
Börsdata ID           0
Company               0
Industry              0
Volume                0
P/FCF                 0
Assets Turn           0
Gross profit         15
Tot. Assets           7
ROC                   0
Market Cap            0
Volatility            0
ROE                   0
FCF                   7
Total Equity          7
Return 3m             0
Return 6m             0
Return 1y             0
Country               0
List                  0
Tick                  0
EV/EBIT               0
Last Report           7
Yahoo                 0
Yield                 0
P/E                   0
P/S                   0
P/B                   0
Asset Growth          0
Info - Report         0
Sector                0
FCFROE                0
GPA                   0
EA ret                0
EA ret std            0
1 Year Volatility     0
1 Year MAD            0
EA Volatility         0
Return 3m Rank        0
Return 6m Rank        0
Return 1y Rank        0
EA ret std Rank 

### Sorting

In [66]:
#### MOMENTUM DATA FRAME
momentum_tmp = data_mom.sort_values(by=['Momentum Rank'])
#### Gestalt Data Frame
gestalt_tmp = data_gestalt.sort_values(by=['Gestalt Rank'])

In [67]:
compact = ['Company', 'List','Tick', 'Gestalt Rank', 'Quality Rank', 'Value Rank', 'Momentum Rank', 'EA ret std', '1 Year Volatility']
compact_mom = ['Company', 'List','Tick','Momentum Rank','EA ret std', '1 Year Volatility']
# numbers of stocks to select for diff strategies

CURRENT_BUYOUTS = ['Aspire Global']


stocks = 30
stocks_mom = 25
## MOMENTUM ##
momentum = momentum_tmp[0:stocks_mom]
Momentum = momentum[compact_mom]


## TRIPLE SORT ##


gestalt=gestalt_tmp[ ~gestalt_tmp['Company'].isin(CURRENT_BUYOUTS)]
gestalt = gestalt[0:stocks]
gestalt = gestalt[compact]


### Print Stock Selection

In [68]:
#gestalt 
th_props = [
  ('font-size', '14px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]
# Set CSS properties for td elements in dataframe
td_props = [('font-size', '12px'), ('text-align', 'center')]
#set caption props
caption_props = [('color', 'black'),('font-size', '22px'),
        ("text-align", "center"),
        ('font-weight', 'bold')]
# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props),
    dict(selector="caption", props=caption_props)
]
gestalt.index = range(1, len(gestalt)+1)
gestalt_view = (gestalt.style.apply(lambda x: ['background: lightgreen' if x.name in range(1, 15+1) 
                              else '' for i in x], axis=1).set_table_styles(styles)
                 .format({'Gestalt Rank': "{:.0f}",'Quality Rank': "{:.0f}", 'Value Rank': "{:.0f}",
                          'Momentum Rank': "{:.0f}",
                          'EA ret std': "{:.1f}",'1 Year Volatility': "{:.1%}"})
                 .set_caption("Gestalt"))
gestalt_view

,Company,List,Tick,Gestalt Rank,Quality Rank,Value Rank,Momentum Rank,EA ret std,1 Year Volatility
1,BE Group,Small Cap,BEGR,1,40,27,3,2.3,52.8%
2,Poolia,Small Cap,POOL B,2,17,46,17,0.5,47.5%
3,Björn Borg,Small Cap,BORG,3,22,49,14,3.2,41.3%
4,B3 Consulting,Small Cap,B3,4,15,71,1,4.7,43.0%
5,Nilörngruppen,Small Cap,NIL B,6,9,76,7,1.8,41.7%
6,Clas Ohlson,Mid Cap,CLAS B,6,20,56,16,3.1,36.1%
7,SSAB B,Large Cap,SSAB B,7,84,6,23,0.3,36.0%
8,New Wave,Mid Cap,NEWA B,8,56,28,31,3.8,45.6%
9,Africa Oil,Mid Cap,AOI,9,100,13,6,1.0,43.0%
10,TietoEVRY,Large Cap,TIETOS,10,51,7,67,1.5,21.5%


In [78]:
th_props = [
  ('font-size', '14px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]
# Set CSS properties for td elements in dataframe
td_props = [('font-size', '12px'), ('text-align', 'center')]
#set caption props
caption_props = [('color', 'black'),('font-size', '22px'),
        ("text-align", "center"),
        ('font-weight', 'bold')]
# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props),
    dict(selector="caption", props=caption_props)
]
Momentum.index = range(1,len(Momentum)+1)
Momentum_view = (Momentum.style.apply(lambda x: ['background: lightgreen' if x.name in range(1,10+1) 
                              else '' for i in x], axis=1).set_table_styles(styles).format({'EA ret std': "{:.1f}",'1 Year Volatility': "{:.1%}"})
                 .set_caption("Focused Momentum"))
Momentum_view

,Company,List,Tick,Momentum Rank,EA ret std,1 Year Volatility
1,Arion Banki,Large Cap,ARION SDB,1.000000,1.6,30.5%
2,Axfood,Large Cap,AXFO,2.000000,3.6,15.9%
3,ALM Equity,First North,ALM,3.000000,3.6,32.6%
4,AstraZeneca,Large Cap,AZN,4.000000,1.2,23.6%
5,B3 Consulting,Small Cap,B3,5.000000,4.7,43.0%
6,Boliden,Large Cap,BOL,6.000000,1.4,30.7%
7,Prevas,Small Cap,PREV B,7.000000,3.6,40.4%
8,Stora Enso R,Large Cap,STE R,8.000000,1.7,25.4%
9,Getinge,Large Cap,GETI B,9.000000,2.0,29.6%
10,Africa Oil,Mid Cap,AOI,10.000000,1.0,43.0%


## Reversed Ranks

In [13]:
stocks = 20
## MOMENTUM ##
momentum = momentum_tmp[0:stocks]
Momentum = momentum[compact_mom]


## TRIPLE SORT ##
gestalt = gestalt_tmp[0:stocks]
gestalt = gestalt[compact]

neg_momentum = momentum_tmp.tail(stocks)[::-1][compact_mom]
neg_gestalt = gestalt_tmp.tail(stocks)[::-1][compact]



In [51]:
#neg_momentum

In [50]:
#neg_gestalt

In [16]:
save = "YES"
if save == "YES":
    data_date  = re.search(r'\d{4}-\d{2}-\d{2}', file_name)
    folder = '../clean_equity_data'
    exp_file_name = "rank_data_{}".format(data_date.group(0))+ ".csv"
    data_tmp.to_csv(os.path.join(folder, exp_file_name))

In [49]:
company('NO', gestalt_tmp)[compact]

,Company,List,Tick,Gestalt Rank,Quality Rank,Value Rank,Momentum Rank,EA ret std,1 Year Volatility
302,NOTE,Mid Cap,NOTE,104.0,107.5,227.0,31.0,0.387972,0.526093


In [32]:
gestalt_tmp[['Momentum Rank', 'Quality Rank', 'Value Rank', 'Gestalt Rank']].corr(method = 'spearman')

,Momentum Rank,Quality Rank,Value Rank,Gestalt Rank
Momentum Rank,1.000000,0.388999,0.349949,0.721533
Quality Rank,0.388999,1.000000,0.639158,0.840646
Value Rank,0.349949,0.639158,1.000000,0.830110
Gestalt Rank,0.721533,0.840646,0.830110,1.000000
